In [ ]:
import sys
import os
import numpy as np
import scipy.signal as sp
import processing as pr
import matplotlib.pyplot as plt
import pandas as pd
from ruamel.yaml import YAML as ym
import datetime
import copy
import re
import scipy.fft
import pickle

# Use this for interactive graphs - nice for exploring
#%matplotlib widget

%matplotlib inline

import mplcursors

sys.path.append("../preprocessing")
from generate_chirp import generate_chirp

# Widgets are only needed if you want to use the interactive plot at the end
# Installation instructions: https://ipywidgets.readthedocs.io/en/latest/user_install.html
import ipywidgets as widgets

In [ ]:
# 9-1 Vatnajokull Day 2
prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_085756"
prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_090136"
prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_090856"
prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_091051"
prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_091247"
prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_091437"
prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_091643"

prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_095129" # Flight 1 -- possibly too aggressive PRF
px4_logs = "../../px4_logs/2022-09-01/16_40_36/16_40_36_"

# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_101053" # 350 MHz center for anna
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_102013" # back to regular -- rect window now
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_102245"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_102720"
# prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_102935"

prefix = "../../radar_data/20220901-vatnajokull-day2/20220901_105725" # Flight 2
px4_logs = "../../px4_logs/2022-09-01/17_43_54/17_43_54_"

# Vatnajokull Day 3 -- Wind, stuck in cabin

# Vatnajokull Day 4

prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_030152"
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_033000"
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_035332" # Flight 1 - V2 to V1 transect
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_045320" # 350 mhz center, longer pulse
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_045646" # 3 db more tx power
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_045933" # another 3db
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_062538" # 2 more db
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_062823" # 150 us pulse
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_063113" # same thing, more pulses
prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_064218" # final check

prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_070453" # Flight 2
px4_logs = "../../px4_logs/2022-09-0313_49_57/13_49_57/13_49_57_"

#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_082804" # 20 us pulse length, faster prf
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083111" # faster prf
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083343" # tried blackman window
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083526" # back to rect
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083713" # back to 390 mhz
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_083842" # blackman win
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_084054" # 3 db more
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_084251"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_084933"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_085122"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_085245"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_085723"
#prefix = "../../radar_data/20220903-vatnajokull-day4/20220903_091901" # Flight 3

# Vatnajokull Day 5 -- SKF6
#prefix = "../../radar_data/20220904-vatnajokull-day5/20220904_032849"
#prefix = "../../radar_data/20220904-vatnajokull-day5/20220904_041423" # Flight 1.5 (forgot to turn on radar for first one)
#prefix = "../../radar_data/20220904-vatnajokull-day5/20220904_054559" # Flight 2 (the long one)



#px4_logs = None

yaml_file = prefix + "_config.yaml"
bin_file = prefix + "_rx_samps.bin"
log_file = prefix + "_uhd_stdout.log"

# For PX4 logs: They must be converted to a set of CSV files with pyulog
# For example: `ulog2csv 17_39_45.ulg -o 17_39_45/`

# Thomas's system at 56 MHz sample rate - zero_sample_idx = 159
zero_sample_idx = 159

In [ ]:
# Initialize Constants
yaml = ym()
with open(yaml_file) as stream:
    config = yaml.load(stream)
    sample_rate = config["PLOT"]["sample_rate"]    # Hertz
    sig_speed = config["PLOT"]["sig_speed"] / np.sqrt(3.17) ## TODO ICE
    
    rx_len_samples = int(config['CHIRP']['rx_duration'] * config['GENERATE']['sample_rate'])
    
rx_samps = bin_file

config_blackman_window = copy.deepcopy(config)
config_blackman_window['GENERATE']['window'] = 'blackman'

# Read and plot RX/TX
# This cell loads all of the data - it can take a while with a large file. You don't need to re-run this cell if you only change n_stack
_, tx_sig = generate_chirp(config)
_, tx_sig_blackman_window = generate_chirp(config_blackman_window)
pr.plotChirpVsTime(tx_sig, 'Transmitted Chirp', sample_rate)
print(f"len(tx_sig): {len(tx_sig)}")

t0 = datetime.datetime.now()

max_chunk_size = int(5e8)
max_seconds_to_load = 60*20
load_start_seconds = 0

max_file_size_bytes = rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*8*max_seconds_to_load
load_start_bytes = rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*8*load_start_seconds

file_size_bytes = os.path.getsize(rx_samps) - load_start_bytes
if file_size_bytes > max_file_size_bytes:
    print(f"WARNING: File is {file_size_bytes/(2**30):.2f} GB ({file_size_bytes / (rx_len_samples*int(1/config['CHIRP']['pulse_rep_int'])*2):.2f} seconds). Only loading the first {max_seconds_to_load} seconds.")
    file_size_bytes = max_file_size_bytes

rx_sig = np.zeros((file_size_bytes//8,), dtype=np.csingle)
for start_offset in np.arange(0, file_size_bytes, max_chunk_size, dtype=np.int64):
    if start_offset + max_chunk_size > file_size_bytes:
        rx_sig[(start_offset//8):] = pr.extractSig(rx_samps, count=file_size_bytes-start_offset, offset=load_start_bytes+start_offset)
    else:
        rx_sig[(start_offset//8):((start_offset//8)+(max_chunk_size//2))] = pr.extractSig(rx_samps, count=max_chunk_size, offset=load_start_bytes+start_offset)

print(datetime.datetime.now() - t0)

In [ ]:
## Reshape data

n_rxs = len(rx_sig) // rx_len_samples
rx_sig_reshaped = np.transpose(np.reshape(rx_sig, (n_rxs, rx_len_samples), order='C'))
errors_removed = False # Keep track of if the cell removing errors from rx_sig_reshaped has already been run

print(f"len(rx_sig): {len(rx_sig)}")
print(f"n_rxs: {n_rxs}")
print(f"rx_sig shape: {np.shape(rx_sig)}")
print(f"rx_sig_reshaped shape: {np.shape(rx_sig_reshaped)}")

# Extract log information about errors and start timestamp

errors = None
start_timestamp = None

if os.path.exists(log_file):
    errors = {}
    
    log_f = open(log_file, 'r')
    log = log_f.readlines()
    
    for idx, line in enumerate(log):
        if "Receiver error:" in line:
            error_code = re.search("(?:Receiver error: )([\w_]+)", line).groups()[0]
            old_style_regex_serach = re.search("(?:Scheduling chirp )([\d]+)", log[idx-1])
            if old_style_regex_serach is not None:
                chirp_idx = int(re.search("(?:Scheduling chirp )([\d]+)", log[idx-1]).groups()[0])
            else:
                chirp_idx = int(re.search("(?:Chirp )([\d]+)", line).groups()[0])
            errors[chirp_idx] = error_code
            if error_code != "ERROR_CODE_LATE_COMMAND":
                print(f"WARNING: Uncommon error in the log: {error_code} (on chirp {chirp_idx})")
                print(f"Full message: {line}")
        if "[START]" in line:
            start_timestamp = float(re.search("(?:\[)([\d]+\.[\d]+)", line).groups()[0])
else:
    print(f"WARNING: No log file found. This is fine, but checks for error codes will be disabled.")
    print(f"(Looking for a log file in: {log_file})")

# Handle errors

# Choose what to do with chirps with a reported error (usually ERROR_CODE_LATE_COMMAND)
# Options are:
# None - do nothing
# 'zeros' - replace with zeros
# 'remove' - remove them from the rx_sig_reshaped array
error_behavior = None

if errors is None:
    if error_behavior is not None:
        print(f"WARNING: Requested doing something with errors but no log file was loaded. Defaulting to doing nothing.")
elif error_behavior == 'zeros':
    error_idxs = np.array(list(errors.keys()))
    rx_sig_reshaped[:,error_idxs] = 0
elif error_behavior == 'remove':
    if errors_removed:
        print(f"WARNING: Error chirps already removed since rx_sig_reshaped created. Not doing again.")
    else:
        error_idxs = np.array(list(errors.keys()))
        all_idxs = np.arange(rx_sig_reshaped.shape[1])
        keep_idxs = [x for x in all_idxs if x not in error_idxs]

        rx_sig_reshaped = rx_sig_reshaped[:, keep_idxs]
        n_rxs = len(keep_idxs)
        errors_removed = True
    
print(f"n_rxs: {n_rxs}")
print(f"rx_sig_reshaped shape: {np.shape(rx_sig_reshaped)}")
print(f"Extracted start timestamp: {start_timestamp}")

In [ ]:
slow_time = np.linspace(0, config['CHIRP']['pulse_rep_int']*config['CHIRP']['num_presums']*n_rxs, np.shape(rx_sig_reshaped)[1])

for idx in errors:
    slow_time[idx+1:] += config['CHIRP']['pulse_rep_int'] * 3

# TODO: Should play with this more to see if it correctly accounts for drift to to errors

#plt.plot(np.diff(slow_time))

In [ ]:
chirp_idx = 150
fig, axs = pr.plotChirpVsTime(rx_sig_reshaped[:,chirp_idx], 'Recieved Chirp', sample_rate)
axs[0].text(0, 1.1, prefix.split("/")[-1] + "\n" + f"chirp_idx = {chirp_idx}", horizontalalignment='left', verticalalignment='center', transform=axs[0].transAxes)
plt.savefig(prefix + "_rx_chirp.png")

In [ ]:
def plot_radargram(n_stack=15, start_time_s=0, duration_s=-1, upsampling=1, figsize=None, vmin=-70, vmax=-40, ylims=(65, 15), surface_dist=None):

    if duration_s < 0:
        duration_s = np.max(slow_time) - start_time_s

    start_idx = np.argmin(np.abs(slow_time - start_time_s))
    end_idx = np.argmin(np.abs(slow_time - (start_time_s + duration_s)))
    actual_duration_s = slow_time[end_idx] - slow_time[start_idx]

    rx_sig_cropped = rx_sig_reshaped[:, start_idx:end_idx]

    corr_sig = tx_sig_blackman_window

    if upsampling > 1:
        corr_sig = scipy.signal.resample_poly(corr_sig, upsampling, 1)

    xcorr_results = np.zeros((((rx_len_samples*upsampling)-len(corr_sig))+1, np.shape(rx_sig_cropped)[1]//n_stack), dtype=np.csingle)

    if surface_dist is not None:
        surface_dist_plot = surface_dist[np.arange(start_idx, end_idx-n_stack, n_stack)]
    
    slow_time_plot = slow_time[np.arange(start_idx, end_idx-n_stack, n_stack)]
    distance_to_reflector = np.linspace(0, np.shape(xcorr_results)[0]/(sample_rate*upsampling), np.shape(xcorr_results)[0]) * sig_speed / 2
    distance_to_reflector = distance_to_reflector - distance_to_reflector[zero_sample_idx*upsampling]

    for res_idx in range(np.shape(xcorr_results)[1]):
        stacked = np.mean(rx_sig_cropped[:,res_idx*n_stack:(res_idx+1)*n_stack], axis=1)
        if upsampling > 1:
            stacked = scipy.signal.resample_poly(stacked, upsampling, 1)

        if surface_dist is not None:
            surf_idx_offset = np.argmin(np.abs(distance_to_reflector - surface_dist_plot[res_idx])) - np.argmin(np.abs(distance_to_reflector))
            xcorr_results[:-surf_idx_offset, res_idx] = sp.correlate(stacked, corr_sig, mode='valid', method='auto')[surf_idx_offset:] / np.sum(np.abs(corr_sig)**2)
        else:
            xcorr_results[:, res_idx] = sp.correlate(stacked, corr_sig, mode='valid', method='auto') / np.sum(np.abs(corr_sig)**2)
        
    if figsize is None:
        figsize = (duration_s/10, 5)
            
    fig, ax = plt.subplots(1,1, figsize=figsize, facecolor='white')

    

    return_power = 20*np.log10(np.abs(xcorr_results))
    p = ax.pcolormesh(slow_time_plot, distance_to_reflector, return_power, shading='auto', cmap='inferno', vmin=vmin, vmax=vmax)
    clb = fig.colorbar(p, ax=ax)
    clb.set_label('Power [dB]')
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Distance to reflector [m]')

    ax.set_ylim(ylims[0], ylims[1])
    ax.set_xlim(start_time_s,start_time_s+duration_s)

    ax.text(0, 1.05, prefix.split("/")[-1] + "\n" + f"n_stack = {n_stack}", horizontalalignment='left', verticalalignment='center', transform=ax.transAxes)

    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, clb.ax.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels() + clb.ax.get_yticklabels()): #ax.get_legend().get_texts()
        item.set_fontsize(18)
        item.set_fontfamily('sans-serif')
        
    fig.tight_layout()

    return fig, ax, {'slow_time': slow_time_plot, 'distance_to_reflector': distance_to_reflector, 'return_power': return_power}

In [ ]:
fig, ax, _ = plot_radargram(n_stack=20, start_time_s=0, duration_s=-1, upsampling=2, figsize=(15, 5), vmin=-90, vmax=-35, ylims=(100, 15))

In [ ]:
if px4_logs is not None:
    # Get UTC time from GPS
    df = pd.read_csv(px4_logs + "sensor_gps_0.csv")
    px4_timestamp_offset_to_utc = (df['time_utc_usec'].at[0] / 1e6) - (df['timestamp'].at[0] / 1e6)

    slow_time_unix = slow_time + start_timestamp

    def interp_log_to_radar(df, key):
        ts = df['timestamp']*1e-6 + px4_timestamp_offset_to_utc

        interp_fn = scipy.interpolate.interp1d(ts, df[key], kind='linear', bounds_error=False)
        return interp_fn(slow_time_unix)

    px4_logdata = {'time_s': slow_time}

    # Load distance sensor data
    df_distance_sensor = pd.read_csv(px4_logs + "distance_sensor_0.csv")
    df_distance_sensor['dist_filt'] = df_distance_sensor['current_distance']
    df_distance_sensor['dist_filt'].iloc[df_distance_sensor['dist_filt'] > 120] = None
    scipy.signal.medfilt
    px4_logdata['dist'] = interp_log_to_radar(df_distance_sensor, 'dist_filt')

    df_position = pd.read_csv(px4_logs + "estimator_global_position_0.csv")
    df_position['alt_rel'] = df_position['alt'] - df_position['alt'].min()
    for k in ['lat', 'lon', 'alt_rel']:
        px4_logdata[k] = interp_log_to_radar(df_position, k)

    df_px4 = pd.DataFrame(px4_logdata, index=slow_time)

    df_px4

In [ ]:
if px4_logs is not None:
    fig, ax, plot_data = plot_radargram(n_stack=15, start_time_s=50, duration_s=650, upsampling=3, figsize=(15, 5), vmin=-70, vmax=-40, ylims=(80, 15))
    ax.scatter(df_px4['time_s'], df_px4['dist'] / np.sqrt(3.17), c='white', s=1)
    #ax.plot(df_px4['time_s'], d_interp, c='white')
    #ax.plot(df_px4['time_s'], df_px4['alt_rel'] / np.sqrt(3.17))

In [ ]:
if px4_logs is not None:
    fig, ax = plt.subplots(figsize=(10,10))
    ax.plot(df_px4['lat'], df_px4['lon'])

    pres_points = [(64.39152, -17.32642), (64.38061, -17.36976)]
    ax.scatter([x[0] for x in pres_points], [x[1] for x in pres_points], c='red')

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.scatter(df_px4['time_s'], df_px4['dist'] / np.sqrt(3.17), c='blue', s=1)

d = df_px4['dist'].to_numpy() / np.sqrt(3.17)

laser_delta = np.diff(d) / np.diff(df_px4['time_s'].to_numpy())
laser_delta = np.concatenate([laser_delta, [0]])
print(np.shape(laser_delta))

d[np.isnan(laser_delta)] = np.nan
d[np.abs(laser_delta) > 2] = np.nan

d[d > 50] = np.nan
d[d < 20] = np.nan

interp = scipy.interpolate.interp1d((df_px4['time_s'].to_numpy())[~np.isnan(d)], d[~np.isnan(d)], kind='cubic', bounds_error=False)
d_interp = interp(df_px4['time_s'])

ax.scatter(df_px4['time_s'], d, c='red', s=1)
ax.plot(df_px4['time_s'], d_interp, c='orange')

#ax.scatter((df_px4['time_s'].to_numpy())[:-1], np.diff(d) / np.diff(df_px4['time_s'].to_numpy()))


In [ ]:
fig, ax, plot_data = plot_radargram(n_stack=15, start_time_s=350, duration_s=100, upsampling=10, figsize=(15, 5), vmin=-65, vmax=-45, ylims=(60, 15))
fig, ax, plot_data = plot_radargram(n_stack=15, start_time_s=350, duration_s=100, upsampling=10, figsize=(15, 5), vmin=-65, vmax=-45, ylims=(35, -10), surface_dist=d_interp)

In [ ]:
np.argmin(np.abs(plot_data['distance_to_reflector'])), np.argmin(np.abs(plot_data['distance_to_reflector'] - 60))

In [ ]:
s = ret_pwr[318:398,1000]
peaks, _ = scipy.signal.find_peaks(s, prominence=(10, None), height=-60)
plt.plot(s)
plt.scatter(peaks, s[peaks])

In [ ]:
help(scipy.signal.find_peaks)

In [ ]:
ret_pwr = plot_data['return_power']
peak_idxs = np.zeros((np.shape(ret_pwr)[1],), dtype=int) * np.nan

peak_dists = np.zeros(peak_idxs.shape) * np.nan

for idx in range(ret_pwr.shape[1]):
    peaks, _ = scipy.signal.find_peaks(ret_pwr[318:398, idx], prominence=(10, None), height=-60)
    if len(peaks) > 0:
        peak_idxs[idx] = peaks[0] + 318
        peak_dists[idx] = plot_data['distance_to_reflector'][peaks[0] + 318]



plt.plot(peak_dists)

In [ ]:
peak_dists_filt = scipy.signal.medfilt(peak_dists, 11)
plt.plot(peak_dists_filt)

In [ ]:
peak_idxs

In [ ]:
# Dump interpolated data to a pickle file

px4_interp = {'time_s': plot_data['slow_time']}

for k in df_px4:
    interp = scipy.interpolate.interp1d(df_px4['time_s'], df_px4[k], kind='linear', bounds_error=False)
    px4_interp[k] = interp(plot_data['slow_time'])

df_px4_interp = pd.DataFrame(px4_interp)

start_idx, end_idx = np.argmin(np.abs(plot_data['distance_to_reflector'] - -10)), np.argmin(np.abs(plot_data['distance_to_reflector'] - 100))

data = {
    'return_power': plot_data['return_power'][start_idx:end_idx, :], # (2241 by 2666) where 2241 is the number of range bins and 2666 is the number of timesteps
    'slow_time': plot_data['slow_time'], # 2666 -- relative timestamps in seconds
    'distance_to_reflector': plot_data['distance_to_reflector'][start_idx:end_idx], # 2441 -- one-way distance to a reflector assuming propagation velocity in ice (speed of light / sqrt(3.17))
    'lat': df_px4_interp['lat'].to_numpy(), # latitude
    'lon': df_px4_interp['lon'].to_numpy(), # longitude
    'alt_rel': df_px4_interp['alt_rel'].to_numpy(),
    'dist': df_px4_interp['dist'].to_numpy()
}

with open('20220903_flight2_full.pickle', 'wb') as f:
   pickle.dump(data, f)

# Code to read:
# import pickle
# import numpy as np

# with open('20220901_flight2_full.pickle', 'rb') as f:
#     data = pickle.load(f)

# for k in data:
#     print(f"[{k}] shape: {np.shape(data[k])}")


In [ ]:
with open('20220901_flight2.pickle', 'rb') as f:
    data = pickle.load(f)

data